In [37]:
# REFERENCES
#
#     Tutorial : http://machinelearningmastery.com/naive-bayes-classifier-scratch-python/
#         Note : Reference is explain in detail better than this but it use Python 2
#                In this tutorial implement in Python 3 (change just print.format() and dict.iteritems())
#     Naive Bayes : https://en.wikipedia.org/wiki/Naive_Bayes_classifier
#     Gaussian Probability Density / Normal distribution : 
#         https://en.wikipedia.org/wiki/Normal_distribution
#     Dataset : https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes

# Handle Data: Load the data from CSV file and split it into training and test datasets.
import csv
import random

def loadCsv(file):
    lines = csv.reader(open(file, "r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

### Test
# dataset = loadCsv(datasetPath)
# print("Loaded data file " + datasetPath + " with " + str(len(dataset)) + " rows")

#     split train test
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

### Test
# dataset = [[1],[2],[3],[4],[5]]
# splitRatio = 0.67
# train, test = splitDataset(dataset, splitRatio)
# print(train, test)

# Summarize Data: summarize the properties in the training dataset so that we can calculate probabilities and make predictions.
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if(vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

### Test
# dataset = [[1,20,1], [2,21,0], [3,22,1]]
# separated = separateByClass(dataset)
# print("Separated instances: " + str(separated))

import math
def mean(numbers):
    return sum(numbers)/float(len(numbers))
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg, 2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

### Test
# numbers = [1,2,3,4,5]
# print('Summary of ' + str(numbers) + ': mean=' + str(mean(numbers)) + ', stdev=' + str(stdev(numbers)))

def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

### Test
# dataset = [[1,20,0], [2,21,1], [3,22,0]]
# summary = summarize(dataset)
# print('Attribute summaries: ' + str(summary))

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

### Test
# dataset = [[1,20,1], [2,21,0], [3,22,1], [4,22,0]]
# summary = summarizeByClass(dataset)
# print('Summary by class value: ' + str(summary))

# Make a Prediction: Use the summaries of the dataset to generate a single prediction.
import math
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean, 2)/(2*math.pow(stdev, 2))))
    return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent

### Test
# x = 71.5
# mean = 73
# stdev = 6.2
# probability = calculateProbability(x, mean, stdev)
# print('Probability of belonging to this class: ' + str(probability))

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

### Test
# summaries = {0:[(1, 0.5)], 1:[(20, 5.0)]}
# inputVector = [1.1, '?']
# probabilities = calculateClassProbabilities(summaries, inputVector)
# print('Probabilities for each class: ' + str(probabilities))

def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

### Test
# summaries = {'A':[(1, 0.5)], 'B':[(20, 5.0)]}
# inputVector = [1.1, '?']
# result = predict(summaries, inputVector)
# print('Prediction: ' + result)

# Make Predictions: Generate predictions given a test dataset and a summarized training dataset.
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

### Test
# summaries = {'A':[(1, 0.5)], 'B':[(20, 5.0)]}
# testSet = [[1.1, '?'], [19.1, '?']]
# predictions = getPredictions(summaries, testSet)
# print('Predictions: ' + str(predictions))

# Evaluate Accuracy: Evaluate the accuracy of predictions made for a test dataset as the percentage correct out of all predictions made.
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct / float(len(testSet))) * 100.0

### Test
# testSet = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
# predictions = ['a', 'a', 'a']
# accuracy = getAccuracy(testSet, predictions)
# print('Accuracy: ' + str(accuracy))

# Main
def main():
    filename = 'dataset/pima-indians-diabetes.data.csv'
    splitRatio = 0.67
    dataset = loadCsv(filename)
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('Split ' + str(len(dataset)) + ' rows into train=' + str(len(trainingSet)) + ' and test=' + str(len(testSet)) + ' rows')
    
    # prepare model
    summaries = summarizeByClass(trainingSet)
    
    # test model
    predictions = getPredictions(summaries, testSet)
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy : ' + str(accuracy))

main()

Split 768 rows into train=514 and test=254 rows
Accuracy : 71.65354330708661
